*This notebook is based on an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

# Data Indexing and Selection

Here we'll look at similar means of accessing and modifying values in Pandas ``Series`` and ``DataFrame`` objects.
If you have used the NumPy patterns, the corresponding patterns in Pandas will feel very familiar, though there are a few quirks to be aware of.

We'll start with the more salient case of the `DataFrame` and move onto the simpler case of the `Series`.

In [ ]:
import pandas as pd
import numpy as np

## Data Selection in DataFrame

Recall that a ``DataFrame`` acts in many ways like a two-dimensional, structured array, or matrix.  You can think of each row as indicated by its index (explicitly or implicitly named) and its column index (also explicitly or implicitly named).

### Accessing a single column

The first analogy we will consider is the ``DataFrame`` as a dictionary of related ``Series`` objects.
Let's return to our example of areas and populations of states:

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/allisonhorst/palmerpenguins/master/inst/extdata/penguins.csv')
data.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


The individual ``Series`` that make up the columns of the ``DataFrame`` can be accessed via dictionary-style indexing of the column name:

In [ ]:
# access by dictionary-style indexing
data['species']

0         Adelie
1         Adelie
2         Adelie
3         Adelie
4         Adelie
         ...    
339    Chinstrap
340    Chinstrap
341    Chinstrap
342    Chinstrap
343    Chinstrap
Name: species, Length: 344, dtype: object

Equivalently, we can use attribute-style access with column names that are strings:

In [ ]:
# access with attribute-style access
data.species

0         Adelie
1         Adelie
2         Adelie
3         Adelie
4         Adelie
         ...    
339    Chinstrap
340    Chinstrap
341    Chinstrap
342    Chinstrap
343    Chinstrap
Name: species, Length: 344, dtype: object

This attribute-style column access actually accesses the exact same object as the dictionary-style access:

In [ ]:
# Check to see if access is the same
data.species is data['species']

True

Though this is a useful shorthand, keep in mind that it does not work for all cases!
For example, if the column names are not strings, or if the column names conflict with methods of the ``DataFrame``, this attribute-style access is not possible.
For example, the ``DataFrame`` has a ``pop()`` method, so ``data.pop`` will point to this rather than the ``"pop"`` column if there were a `population` (`pop` for short) column in this dataframe:

In [ ]:
# Challenges of attribute-style access
data.pop is data['pop']

In particular, you should avoid the temptation to try column assignment via attribute (i.e., use ``data['pop'] = z`` rather than ``data.pop = z``).

Some vectorized operations can be completed as well - applying to all of the elements of the selected column.

In [ ]:
data['bill_ld_ratio'] = data['bill_length_mm'] / data['bill_depth_mm']
data

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bill_ld_ratio
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007,2.090909
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007,2.270115
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007,2.238889
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007,1.901554
...,...,...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009,2.818182
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009,2.403315
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009,2.725275
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009,2.673684


**Memory check:** What was that operation called in the tidyverse that added a column to your data?  Does this look like a similar operation?

### Accessing as row, column: `loc` and `iloc`

As mentioned previously, we can also view the ``DataFrame`` as an enhanced two-dimensional array.
We can examine the raw underlying data array using the ``values`` attribute:

In [ ]:
# get underlying values as numpy array
data.values

array([['Adelie', 'Torgersen', 39.1, ..., 'male', 2007,
        2.0909090909090913],
       ['Adelie', 'Torgersen', 39.5, ..., 'female', 2007,
        2.270114942528736],
       ['Adelie', 'Torgersen', 40.3, ..., 'female', 2007,
        2.238888888888889],
       ...,
       ['Chinstrap', 'Dream', 49.6, ..., 'male', 2009,
        2.7252747252747254],
       ['Chinstrap', 'Dream', 50.8, ..., 'male', 2009,
        2.6736842105263157],
       ['Chinstrap', 'Dream', 50.2, ..., 'female', 2009,
        2.6844919786096257]], dtype=object)

For array-style indexing, we need another convention.
Here Pandas again uses the ``loc`` and ``iloc`` indexers.  **Recall that in our previous exploration of series, explicit indices can be numerical, convoluting whether you're selecting by implicit or explicit index.  These two methods can disambiguate the approach.**

### `iloc`
Think of `iloc` as a reference to the _integer_ location, or _implicit_ index, or by position.  Using the ``iloc`` indexer, we can index the underlying array and the ``DataFrame`` index and column labels are maintained in the result.  The indexer expects the row position followed by the column position.

In [ ]:
data

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bill_ld_ratio
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007,2.090909
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007,2.270115
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007,2.238889
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007,1.901554
...,...,...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009,2.818182
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009,2.403315
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009,2.725275
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009,2.673684


In [ ]:
# index the Dataframe by position, row 0, column 0
print(data.iloc[0,0])

#index the dataframe; find row 0, column1
data.iloc[0,1]

Adelie


'Torgersen'

In [ ]:
# slice the dataframe to get the first 3 rows and the first 2 columns
data.iloc[:3, :2]

### `loc`

Similarly, using the ``loc`` indexer we can index the underlying data in an array-like style but using the _explicit index_ and column names.  Note that since this has no explicit index, we use the implicit index given to the same effect.

In [ ]:
#Get the first row
data.loc[0]

species                 Adelie
island               Torgersen
bill_length_mm            39.1
bill_depth_mm             18.7
flipper_length_mm          181
body_mass_g               3750
sex                       male
year                      2007
bill_ld_ratio          2.09091
Name: 0, dtype: object

**Question**.  We just said that these indexers require the format `[row, column]`.  However, it seems like we omitted that here.  What do you think is happening?

**Gotcha!** Look at how this function appears.  It looks _very_ similar to `data['California']`.  However, these behaviors are decidedly different; always remember that `.loc` will expect a _row index_ as the first parameter in the bracket, whereas dictionary-like key-value lookup will expect a column index.  

In [ ]:
# Try to use loc to just access the column
data.loc['species']

KeyError: 'species'

In [ ]:
# Correctly use loc to access the column
data.loc[:, 'species']

0         Adelie
1         Adelie
2         Adelie
3         Adelie
4         Adelie
         ...    
339    Chinstrap
340    Chinstrap
341    Chinstrap
342    Chinstrap
343    Chinstrap
Name: species, Length: 344, dtype: object

In [ ]:
# Can also slice
data.loc[:10, :'year'] #explicit indices include the endpoint

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007
6,Adelie,Torgersen,38.9,17.8,181.0,3625.0,female,2007
7,Adelie,Torgersen,39.2,19.6,195.0,4675.0,male,2007
8,Adelie,Torgersen,34.1,18.1,193.0,3475.0,NaN,2007
9,Adelie,Torgersen,42.0,20.2,190.0,4250.0,NaN,2007


### Try it yourself!!
Using the `data` dataframe above, try to do the following selections:
1. Select the `bill_depth_mm` column using `iloc`, `loc`, and dictionary indexing.
2. Select only the 1 and 5 rows with all columns.
3. Select the `species` and `island` columns with all rows.
4. Select rows 2, 4, and 6 years.
5. **Bonus**: Repeat #4, but reverse the order (e.g., dataframe returned should have order 6, 4, 2)

**HINT**: When trying to select multiple indices, you can pass in a list of indices (e.g., `data.loc[['John', 'Raheim'], :]`)

In [ ]:
# Answer to 1
print(data['bill_depth_mm'])
print(data.loc[:, 'bill_depth_mm'])
print(data.iloc[:, 3])

0      18.7
1      17.4
2      18.0
3       NaN
4      19.3
       ... 
339    19.8
340    18.1
341    18.2
342    19.0
343    18.7
Name: bill_depth_mm, Length: 344, dtype: float64
0      18.7
1      17.4
2      18.0
3       NaN
4      19.3
       ... 
339    19.8
340    18.1
341    18.2
342    19.0
343    18.7
Name: bill_depth_mm, Length: 344, dtype: float64
0      18.7
1      17.4
2      18.0
3       NaN
4      19.3
       ... 
339    19.8
340    18.1
341    18.2
342    19.0
343    18.7
Name: bill_depth_mm, Length: 344, dtype: float64


In [ ]:
#Answer to 2
display(data.iloc[[1,5]])
data.loc[[1,5], :]

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bill_ld_ratio
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007,2.270115
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007,1.907767


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bill_ld_ratio
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007,2.270115
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007,1.907767


In [ ]:
#Answer to 3
display(data.iloc[:, :2])
data.loc[:, ['species', 'island']]

,species,island
0,Adelie,Torgersen
1,Adelie,Torgersen
2,Adelie,Torgersen
3,Adelie,Torgersen
4,Adelie,Torgersen
...,...,...
339,Chinstrap,Dream
340,Chinstrap,Dream
341,Chinstrap,Dream
342,Chinstrap,Dream


,species,island
0,Adelie,Torgersen
1,Adelie,Torgersen
2,Adelie,Torgersen
3,Adelie,Torgersen
4,Adelie,Torgersen
...,...,...
339,Chinstrap,Dream
340,Chinstrap,Dream
341,Chinstrap,Dream
342,Chinstrap,Dream


In [ ]:
#Answer to 4
display(data.iloc[[2,4,6], 7])
display(data.iloc[2:7:2, 7])
data.loc[2:6:2, 'year']

2    2007
4    2007
6    2007
Name: year, dtype: int64

2    2007
4    2007
6    2007
Name: year, dtype: int64

2    2007
4    2007
6    2007
Name: year, dtype: int64

In [ ]:
#Answer to 5
data.loc[6:2:-2, 'year']

6    2007
4    2007
2    2007
Name: year, dtype: int64

### Masking and fancy indexing

Many different data access patterns can be used within these indexers.
For example, in the ``loc`` indexer we can combine boolean masking and fancy indexing as in the following.  Boolean masking evaluates a certain condition on your dataset, and then returns the true/false value of this condition.  The returned dataset can then be used to filter out the `false` or `true` values.  Additionally fancy indexing can allow you to select multiple columns using a list.

In [ ]:
# Look at the boolean masking result of bill_depth_mm > 17
data['bill_depth_mm'] > 17
data.loc[data['bill_depth_mm'] > 17,:]

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bill_ld_ratio
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007,2.090909
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007,2.270115
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007,2.238889
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007,1.901554
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007,1.907767
...,...,...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009,2.818182
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009,2.403315
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009,2.725275
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009,2.673684


**Question**: How big is this dataframe?  What's up with the indices?

In [ ]:
# Only look at selected columns - island and body_mass_g
data.loc[data['bill_depth_mm'] > 17, ['island', 'body_mass_g']]

,island,body_mass_g
0,Torgersen,3750.0
1,Torgersen,3800.0
2,Torgersen,3250.0
4,Torgersen,3450.0
5,Torgersen,3650.0
...,...,...
339,Dream,4000.0
340,Dream,3400.0
341,Dream,3775.0
342,Dream,4100.0


In [ ]:
# Look at all rows that aren't index 0 (this is valuable when you have indices that are non-unique)
data.loc[data.index != 0]

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bill_ld_ratio
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007,2.270115
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007,2.238889
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007,1.901554
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007,1.907767
...,...,...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009,2.818182
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009,2.403315
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009,2.725275
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009,2.673684


In [ ]:
# Find all the columns that aren't species
data.columns != 'species'
data.loc[:, data.columns != 'species']

,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bill_ld_ratio
0,Torgersen,39.1,18.7,181.0,3750.0,male,2007,2.090909
1,Torgersen,39.5,17.4,186.0,3800.0,female,2007,2.270115
2,Torgersen,40.3,18.0,195.0,3250.0,female,2007,2.238889
3,Torgersen,NaN,NaN,NaN,NaN,NaN,2007,NaN
4,Torgersen,36.7,19.3,193.0,3450.0,female,2007,1.901554
...,...,...,...,...,...,...,...,...
339,Dream,55.8,19.8,207.0,4000.0,male,2009,2.818182
340,Dream,43.5,18.1,202.0,3400.0,female,2009,2.403315
341,Dream,49.6,18.2,193.0,3775.0,male,2009,2.725275
342,Dream,50.8,19.0,210.0,4100.0,male,2009,2.673684


Any of these indexing conventions may also be used to set or modify values; this is done in the standard way that you might be accustomed to from working with NumPy:

In [ ]:
# You can also use these selections to set values too
data.iloc[0, 2] = 91
data

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bill_ld_ratio
0,Adelie,Torgersen,91.0,18.7,181.0,3750.0,male,2007,2.090909
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007,2.270115
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007,2.238889
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007,1.901554
...,...,...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009,2.818182
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009,2.403315
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009,2.725275
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009,2.673684


To build up your fluency in Pandas data manipulation, I suggest spending some time with a simple ``DataFrame`` and exploring the types of indexing, slicing, masking, and fancy indexing that are allowed by these various indexing approaches.

### Try it yourself!!
Using the dataframe `work_data` below, answer the following questions:
1. Which penguins have bill depth > 20mm?  _How many_ penguins have bill depth > 20mm?
2. What are the species of the penguins which have bill depth > 20?  (Find the penguins that have bill depth greater than 20 and then return only the species column)
3. Which penguins have bill depth between 18 and 19mm?  What are their corresponding body masses?  **Hints**:  (1) `&` is the `and` operator compatible with Pandas Series.  (2) Remember operator precedence and how parentheses can help with that!
4. **Bonus**: Which penguins have body masses within 100 grams of the median body mass?  **Hint**: use `np.median`

In [ ]:
# Make sure to execute this cell
work_data = data.copy()

In [ ]:
#Answer to question 1
res = work_data.loc[work_data.loc[:, 'bill_depth_mm'] > 20]
display(res)
print('The total number of states with bill_depth > 20 is: {0}'.format(res.shape[0]))

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bill_ld_ratio
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007,1.907767
9,Adelie,Torgersen,42.0,20.2,190.0,4250.0,NaN,2007,2.079208
13,Adelie,Torgersen,38.6,21.2,191.0,3800.0,male,2007,1.820755
14,Adelie,Torgersen,34.6,21.1,198.0,4400.0,male,2007,1.639810
17,Adelie,Torgersen,42.5,20.7,197.0,4500.0,male,2007,2.053140
19,Adelie,Torgersen,46.0,21.5,194.0,4200.0,male,2007,2.139535
35,Adelie,Dream,39.2,21.1,196.0,4150.0,male,2007,1.857820
49,Adelie,Dream,42.3,21.2,191.0,4150.0,male,2007,1.995283
61,Adelie,Biscoe,41.3,21.1,195.0,4400.0,male,2008,1.957346
85,Adelie,Dream,41.3,20.3,194.0,3550.0,male,2008,2.034483


The total number of states with bill_depth > 20 is: 17


In [ ]:
#Answer to question 2
work_data.loc[work_data.loc[:, 'bill_depth_mm']>20, 'species']

5         Adelie
9         Adelie
13        Adelie
14        Adelie
17        Adelie
19        Adelie
35        Adelie
49        Adelie
61        Adelie
85        Adelie
111       Adelie
114       Adelie
117       Adelie
137       Adelie
287    Chinstrap
307    Chinstrap
313    Chinstrap
Name: species, dtype: object

In [ ]:
# Answer to question 3
work_data.loc[(work_data.loc[:, 'bill_depth_mm'] > 18) & (work_data.loc[:, 'bill_depth_mm'] < 19), ['bill_depth_mm', 'body_mass_g']]

,bill_depth_mm,body_mass_g
0,18.7,3750.0
8,18.1,3475.0
18,18.4,3325.0
20,18.3,3400.0
21,18.7,3600.0
...,...,...
331,18.8,3450.0
334,18.8,3800.0
340,18.1,3400.0
341,18.2,3775.0


In [ ]:
# Answer to question 4
med_mass = np.median(work_data['body_mass_g'].dropna())
display(work_data.loc[(work_data['body_mass_g']<med_mass + 100) & (work_data['body_mass_g']>med_mass - 100)])
work_data.loc[work_data['body_mass_g'].between(med_mass-100, med_mass+100)]

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bill_ld_ratio
53,Adelie,Biscoe,42.0,19.5,200.0,4050.0,male,2008,2.153846
63,Adelie,Biscoe,41.1,18.2,192.0,4050.0,male,2008,2.258242
67,Adelie,Biscoe,41.1,19.1,188.0,4100.0,male,2008,2.151832
79,Adelie,Torgersen,42.1,19.1,195.0,4000.0,male,2008,2.204188
99,Adelie,Dream,43.2,18.5,192.0,4100.0,male,2008,2.335135
115,Adelie,Biscoe,42.7,18.3,196.0,4075.0,male,2009,2.333333
125,Adelie,Torgersen,40.6,19.0,199.0,4000.0,male,2009,2.136842
129,Adelie,Torgersen,44.1,18.0,210.0,4000.0,male,2009,2.450000
137,Adelie,Dream,40.2,20.1,200.0,3975.0,male,2009,2.000000
151,Adelie,Dream,41.5,18.5,201.0,4000.0,male,2009,2.243243


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bill_ld_ratio
23,Adelie,Biscoe,38.2,18.1,185.0,3950.0,male,2007,2.110497
29,Adelie,Biscoe,40.5,18.9,180.0,3950.0,male,2007,2.142857
35,Adelie,Dream,39.2,21.1,196.0,4150.0,male,2007,1.857820
36,Adelie,Dream,38.8,20.0,190.0,3950.0,male,2007,1.940000
49,Adelie,Dream,42.3,21.2,191.0,4150.0,male,2007,1.995283
53,Adelie,Biscoe,42.0,19.5,200.0,4050.0,male,2008,2.153846
63,Adelie,Biscoe,41.1,18.2,192.0,4050.0,male,2008,2.258242
65,Adelie,Biscoe,41.6,18.0,192.0,3950.0,male,2008,2.311111
67,Adelie,Biscoe,41.1,19.1,188.0,4100.0,male,2008,2.151832
73,Adelie,Torgersen,45.8,18.9,197.0,4150.0,male,2008,2.423280


## On Pandas: Query
As we've already seen in previous sections, the power of the PyData stack is built upon the ability of NumPy and Pandas to push basic operations into C via an intuitive syntax: examples are vectorized/broadcasted operations in NumPy, and grouping-type operations in Pandas.
While these abstractions are efficient and effective for many common use cases, they often rely on the creation of temporary intermediate objects, which can cause undue overhead in computational time and memory use.

As of version 0.13 (released January 2014), Pandas includes some experimental tools that allow you to directly access C-speed operations without costly allocation of intermediate arrays.
These are the ``eval()`` and ``query()`` functions, which rely on the [Numexpr](https://github.com/pydata/numexpr) package.
In this notebook we will walk through their use and give some rules-of-thumb about when you might think about using them.

You can rewrite many of the above methods using strings in combination with query if they're operating on the columns of the dataframe.  This helps with speed and performance since temporary intermediate variables are not created, and also helps with readability.  Let's see how this would look below.

In [ ]:
# Answer to 1
work_data.loc[work_data.loc[:, 'bill_depth_mm'] > 20]

## with query
work_data.query('bill_depth_mm > 20')

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bill_ld_ratio
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007,1.907767
9,Adelie,Torgersen,42.0,20.2,190.0,4250.0,NaN,2007,2.079208
13,Adelie,Torgersen,38.6,21.2,191.0,3800.0,male,2007,1.820755
14,Adelie,Torgersen,34.6,21.1,198.0,4400.0,male,2007,1.639810
17,Adelie,Torgersen,42.5,20.7,197.0,4500.0,male,2007,2.053140
19,Adelie,Torgersen,46.0,21.5,194.0,4200.0,male,2007,2.139535
35,Adelie,Dream,39.2,21.1,196.0,4150.0,male,2007,1.857820
49,Adelie,Dream,42.3,21.2,191.0,4150.0,male,2007,1.995283
61,Adelie,Biscoe,41.3,21.1,195.0,4400.0,male,2008,1.957346
85,Adelie,Dream,41.3,20.3,194.0,3550.0,male,2008,2.034483


In [ ]:
# Answer to 2
work_data.loc[work_data.loc[:, 'bill_depth_mm']>20, 'body_mass_g']

#with query
work_data.query('bill_depth_mm > 20')['body_mass_g']

5      3650.0
9      4250.0
13     3800.0
14     4400.0
17     4500.0
19     4200.0
35     4150.0
49     4150.0
61     4400.0
85     3550.0
111    4600.0
114    3900.0
117    3775.0
137    3975.0
287    3775.0
307    4300.0
313    4800.0
Name: body_mass_g, dtype: float64

In [ ]:
# Answer to 3
work_data.loc[(work_data.loc[:, 'bill_depth_mm'] > 18) & (work_data.loc[:, 'bill_depth_mm'] < 19)]

#with query
work_data.query('bill_depth_mm > 18 & bill_depth_mm < 19')

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bill_ld_ratio
0,Adelie,Torgersen,91.0,18.7,181.0,3750.0,male,2007,2.090909
8,Adelie,Torgersen,34.1,18.1,193.0,3475.0,NaN,2007,1.883978
18,Adelie,Torgersen,34.4,18.4,184.0,3325.0,female,2007,1.869565
20,Adelie,Biscoe,37.8,18.3,174.0,3400.0,female,2007,2.065574
21,Adelie,Biscoe,37.7,18.7,180.0,3600.0,male,2007,2.016043
...,...,...,...,...,...,...,...,...,...
331,Chinstrap,Dream,52.2,18.8,197.0,3450.0,male,2009,2.776596
334,Chinstrap,Dream,50.2,18.8,202.0,3800.0,male,2009,2.670213
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009,2.403315
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009,2.725275


In [ ]:
# Answer to 4: use @ to reference variables that are not in the dataframe
med_mass = np.median(work_data['body_mass_g'].dropna())

#with query
work_data.query('body_mass_g > @med_mass - 100 & body_mass_g < @med_mass + 100')

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bill_ld_ratio
53,Adelie,Biscoe,42.0,19.5,200.0,4050.0,male,2008,2.153846
63,Adelie,Biscoe,41.1,18.2,192.0,4050.0,male,2008,2.258242
67,Adelie,Biscoe,41.1,19.1,188.0,4100.0,male,2008,2.151832
79,Adelie,Torgersen,42.1,19.1,195.0,4000.0,male,2008,2.204188
99,Adelie,Dream,43.2,18.5,192.0,4100.0,male,2008,2.335135
115,Adelie,Biscoe,42.7,18.3,196.0,4075.0,male,2009,2.333333
125,Adelie,Torgersen,40.6,19.0,199.0,4000.0,male,2009,2.136842
129,Adelie,Torgersen,44.1,18.0,210.0,4000.0,male,2009,2.450000
137,Adelie,Dream,40.2,20.1,200.0,3975.0,male,2009,2.000000
151,Adelie,Dream,41.5,18.5,201.0,4000.0,male,2009,2.243243


### Additional indexing conventions

There are a couple extra indexing conventions that might seem at odds with the preceding discussion, but nevertheless can be very useful in practice.
First, while *indexing* refers to columns, *slicing* refers to rows:

In [ ]:
# Dictionary-like access allows you to slice on the rows
data[0:5]

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bill_ld_ratio
0,Adelie,Torgersen,91.0,18.7,181.0,3750.0,male,2007,2.090909
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007,2.270115
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007,2.238889
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007,1.901554


Such slices can also refer to rows by number rather than by index:

In [ ]:
# Dictionary-like access also allows you to slice by implicit index
data[1:3]

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bill_ld_ratio
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007,2.270115
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007,2.238889


Similarly, direct masking operations are also interpreted row-wise rather than column-wise:

In [ ]:
# Masking operations are implemented row wise
data[data.bill_depth_mm > 20]

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bill_ld_ratio
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007,1.907767
9,Adelie,Torgersen,42.0,20.2,190.0,4250.0,NaN,2007,2.079208
13,Adelie,Torgersen,38.6,21.2,191.0,3800.0,male,2007,1.820755
14,Adelie,Torgersen,34.6,21.1,198.0,4400.0,male,2007,1.639810
17,Adelie,Torgersen,42.5,20.7,197.0,4500.0,male,2007,2.053140
19,Adelie,Torgersen,46.0,21.5,194.0,4200.0,male,2007,2.139535
35,Adelie,Dream,39.2,21.1,196.0,4150.0,male,2007,1.857820
49,Adelie,Dream,42.3,21.2,191.0,4150.0,male,2007,1.995283
61,Adelie,Biscoe,41.3,21.1,195.0,4400.0,male,2008,1.957346
85,Adelie,Dream,41.3,20.3,194.0,3550.0,male,2008,2.034483


These two conventions are syntactically similar to those on a NumPy array, and while these may not precisely fit the mold of the Pandas conventions, they are nevertheless quite useful in practice.

## Data Selection in Series

As we saw in the previous section, a ``Series`` object acts in many ways like a one-dimensional NumPy array, and in many ways like a standard Python dictionary.
If we keep these two overlapping analogies in mind, it will help us to understand the patterns of data indexing and selection in these arrays.

### Series as dictionary

Like a dictionary, the ``Series`` object provides a mapping from a collection of keys to a collection of values:

In [ ]:
#pandas series for testing
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

In [ ]:
#access a single row
data['b']

We can also use dictionary-like Python expressions and methods to examine the keys/indices and values:

In [ ]:
# Use python expression with Series
'a' in data

In [ ]:
# Access the keys in a dictionary-like way
data.keys()

In [ ]:
# Access the items as sets of tuples
list(data.items())

``Series`` objects can even be modified with a dictionary-like syntax.
Just as you can extend a dictionary by assigning to a new key, you can extend a ``Series`` by assigning to a new index value:

In [ ]:
# Set values by dictionary-like syntax
data['e'] = 1.25
data

This easy mutability of the objects is a convenient feature: under the hood, Pandas is making decisions about memory layout and data copying that might need to take place; the user generally does not need to worry about these issues.

### Series as one-dimensional array

A ``Series`` builds on this dictionary-like interface and provides array-style item selection via the same basic mechanisms as NumPy arrays – that is, *slices*, *masking*, and *fancy indexing*.
Examples of these are as follows:

In [ ]:
# slicing by explicit index
data['a':'c']

In [ ]:
# slicing by implicit integer index
data[0:2]

In [ ]:
# masking
data[(data > 0.3) & (data < 0.8)]

In [ ]:
# fancy indexing
data[['a', 'e']]

Among these, slicing may be the source of the most confusion.
Notice that when slicing with an explicit index (i.e., ``data['a':'c']``), the final index is *included* in the slice, while when slicing with an implicit index (i.e., ``data[0:2]``), the final index is *excluded* from the slice.

### Indexers: loc, iloc, and ix

These slicing and indexing conventions can be a source of confusion.
For example, if your ``Series`` has an explicit integer index, an indexing operation such as ``data[1]`` will use the explicit indices, while a slicing operation like ``data[1:3]`` will use the implicit Python-style index.

In [ ]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

In [ ]:
# explicit index when indexing
data[1]

In [ ]:
# implicit index when slicing
data[1:3]

Because of this potential confusion in the case of integer indexes, Pandas provides some special *indexer* attributes that explicitly expose certain indexing schemes.
These are not functional methods, but attributes that expose a particular slicing interface to the data in the ``Series``.

First, the ``loc`` attribute allows indexing and slicing that always references the explicit index:

In [ ]:
# access using explicit index
data.loc[1]

In [ ]:
# Slice using explicit index
data.loc[1:3]

The ``iloc`` attribute allows indexing and slicing that always references the implicit Python-style index:

In [ ]:
# Accss using the implicit index
data.iloc[1]

In [ ]:
# Slice using the implicit index
data.iloc[1:3]

One guiding principle of Python code is that "explicit is better than implicit."
The explicit nature of ``loc`` and ``iloc`` make them very useful in maintaining clean and readable code; especially in the case of integer indexes, I recommend using these both to make code easier to read and understand, and to prevent subtle bugs due to the mixed indexing/slicing convention.

## What we've learned in this lesson:

1. Indexing and selection of `DataFrames` using:
    - Bracket indexing
    - `.iloc`
    - `.loc`
    - `.query`
2. Quirks of dataframe indexing vs slicing
3. Indexing and selection of `Series` using:
    - Bracket indexing
    - `.iloc`
    - `.loc`